In [6]:
import numpy as np 
import pandas as pd

books = pd.read_csv("Books.csv")
ratings = pd.read_csv("Ratings.csv")

C:\Users\Summer\AppData\Local\Temp\ipykernel_29080\3821214228.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Books.csv")


In [7]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [8]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
df = books.merge(ratings, how="left", on="ISBN" )
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676.0,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385.0,0.0


In [10]:
df.shape

(1032345, 10)

In [11]:
df.isnull().sum()

ISBN                      0
Book-Title                0
Book-Author               1
Year-Of-Publication       0
Publisher                 2
Image-URL-S               0
Image-URL-M               0
Image-URL-L               4
User-ID                1209
Book-Rating            1209
dtype: int64

In [12]:
df[df["Book-Rating"].isnull()]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
65244,044990671X,Bronze Mirror,Jeanne Larsen,1992,Ballantine Books,http://images.amazon.com/images/P/044990671X.0...,http://images.amazon.com/images/P/044990671X.0...,http://images.amazon.com/images/P/044990671X.0...,NaN,NaN
65253,0929587081,Men and Brethren,James Gould Cozzens,1989,Elephant Paperbacks,http://images.amazon.com/images/P/0929587081.0...,http://images.amazon.com/images/P/0929587081.0...,http://images.amazon.com/images/P/0929587081.0...,NaN,NaN
65254,155970117X,"My Father, His Son",Reidar Jonsson,1991,Little Brown &amp; Co,http://images.amazon.com/images/P/155970117X.0...,http://images.amazon.com/images/P/155970117X.0...,http://images.amazon.com/images/P/155970117X.0...,NaN,NaN
65255,0688105408,Starlings Laughing: A Memoir of Africa,June Vendall Clark,1991,Harpercollins,http://images.amazon.com/images/P/0688105408.0...,http://images.amazon.com/images/P/0688105408.0...,http://images.amazon.com/images/P/0688105408.0...,NaN,NaN
124362,0671568485,The STAR TREK THE RETURN,William Shatner,1996,Audioworks,http://images.amazon.com/images/P/0671568485.0...,http://images.amazon.com/images/P/0671568485.0...,http://images.amazon.com/images/P/0671568485.0...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1032326,1874166633,Introducing Nietzsche (Foundations in Children...,Laurence Gane,1998,Natl Book Network,http://images.amazon.com/images/P/1874166633.0...,http://images.amazon.com/images/P/1874166633.0...,http://images.amazon.com/images/P/1874166633.0...,NaN,NaN
1032327,0130897930,Core Web Programming (2nd Edition),Marty Hall,2001,Prentice Hall PTR,http://images.amazon.com/images/P/0130897930.0...,http://images.amazon.com/images/P/0130897930.0...,http://images.amazon.com/images/P/0130897930.0...,NaN,NaN
1032328,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh,1998,Addison-Wesley Professional,http://images.amazon.com/images/P/020130998X.0...,http://images.amazon.com/images/P/020130998X.0...,http://images.amazon.com/images/P/020130998X.0...,NaN,NaN
1032329,2268032019,Petite histoire de la dÃ?Â©sinformation,Vladimir Volkoff,1999,Editions du Rocher,http://images.amazon.com/images/P/2268032019.0...,http://images.amazon.com/images/P/2268032019.0...,http://images.amazon.com/images/P/2268032019.0...,NaN,NaN


In [13]:
df = df.dropna(subset=["Book-Rating", "User-ID", "Book-Author","Publisher","Image-URL-L"])

df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
User-ID                0
Book-Rating            0
dtype: int64

In [14]:
df["Book-Rating"].value_counts()

0.0     647291
8.0      91803
10.0     71225
7.0      66401
9.0      60776
5.0      45355
6.0      31687
4.0       7617
3.0       5118
2.0       2375
1.0       1481
Name: Book-Rating, dtype: int64

In [15]:
#Removing non-Book-Rating from the data sets
df = df[df["Book-Rating"] > 0]
df["Book-Rating"] .value_counts()

8.0     91803
10.0    71225
7.0     66401
9.0     60776
5.0     45355
6.0     31687
4.0      7617
3.0      5118
2.0      2375
1.0      1481
Name: Book-Rating, dtype: int64

In [16]:
df["Book-Title"].value_counts()

The Lovely Bones: A Novel                 707
Wild Animus                               581
The Da Vinci Code                         494
The Secret Life of Bees                   406
The Nanny Diaries: A Novel                393
                                         ... 
Jedi Quest - Path to Truth (Star Wars)      1
Walter's Purple Heart                       1
Trapped Fear Street 51                      1
Heather Takes the Reins                     1
From One to One Hundred                     1
Name: Book-Title, Length: 135564, dtype: int64

In [17]:
rating_counts = pd.DataFrame(df["Book-Title"].value_counts())

In [18]:
rare_books = rating_counts[rating_counts["Book-Title"]< 100].index
common_books = df[~df["Book-Title"].isin(rare_books)]
common_books["Book-Title"].value_counts()

The Lovely Bones: A Novel     707
Wild Animus                   581
The Da Vinci Code             494
The Secret Life of Bees       406
The Nanny Diaries: A Novel    393
                             ... 
Congo                         100
Deception Point               100
Daddy's Little Girl           100
Cause of Death                100
Disclosure                    100
Name: Book-Title, Length: 200, dtype: int64

In [19]:
common_books["User-ID"].value_counts()

11676.0     225
16795.0      81
95359.0      61
60244.0      46
240567.0     42
           ... 
239067.0      1
239468.0      1
242643.0      1
248677.0      1
169893.0      1
Name: User-ID, Length: 16665, dtype: int64

In [20]:
#eliminating those who rate the book under 10
common_books = common_books[common_books["User-ID"].map(common_books["User-ID"].value_counts()) >= 10]
common_books["User-ID"].value_counts()

11676.0     225
16795.0      81
95359.0      61
60244.0      46
240567.0     42
           ... 
232106.0     10
104113.0     10
78834.0      10
74286.0      10
208671.0     10
Name: User-ID, Length: 434, dtype: int64

In [21]:
user_book_df = common_books.pivot_table(index=["User-ID"],
                                        columns=["Book-Title"],values="Book-Rating")

In [34]:
#Randomly selecting user
random_user = 1320

In [23]:
# List books by random user
random_user_df = user_book_df[user_book_df.index == random_user]
random_user_df

Book-Title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,"A Child Called \It\"": One Child's Courage to Survive""",A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),A Map of the World,A Painted House,A Prayer for Owen Meany,...,Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus,"\O\"" Is for Outlaw"""
User-ID,,,,,,,,,,,,,,,,,,,,,


In [24]:
# Find books rated by random user
books_read = random_user_df.columns[random_user_df.notna().any()].tolist()
books_read

[]

In [25]:
len(books_read)

0

In [26]:
books_read_df = user_book_df[books_read]
books_read_df

Book-Title
User-ID
254.0
638.0
2766.0
4017.0
6242.0
...
274004.0
274301.0
276050.0


In [27]:
#Determine how many books each user rating
user_book_count = books_read_df.T.notnull().sum()
user_book_count

User-ID
254.0       0.0
638.0       0.0
2766.0      0.0
4017.0      0.0
6242.0      0.0
           ... 
274004.0    0.0
274301.0    0.0
276050.0    0.0
276165.0    0.0
276231.0    0.0
Length: 434, dtype: float64

In [28]:
# organizing the data set

In [29]:
#Organize the data set
user_book_count = user_book_count.reset_index()
user_book_count.columns = ["User_id", "book_count"]
user_book_count.sort_values("book_count", ascending=False)

,User_id,book_count
0,254.0,0.0
285,184299.0,0.0
296,190925.0,0.0
295,189835.0,0.0
294,188010.0,0.0
...,...,...
141,95932.0,0.0
140,95359.0,0.0
139,94347.0,0.0
138,93092.0,0.0


In [30]:
users_same_book = user_book_count[user_book_count["book_count"]>5]["User_id"]
users_same_book

Series([], Name: User_id, dtype: float64)

In [31]:
#Determine the users with the most similar behavior 
final_df = pd.concat([books_read_df[books_read_df.index.isin(users_same_book)],
                      random_user_df[books_read]])

In [33]:
# correlations for all users
# organizing the data set
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df,columns =["corr"])
corr_df = corr_df.reset_index()
top_users = corr_df[(corr_df["User_id_1"] == random_user)][["User_id-2","corr"]].reset_index(drop=True)

top_users

ValueError: cannot insert User-ID, already exists

In [ ]:
# other users with 60% Rating
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] > 0.06)][
    ["user_id_2", "corr"]].reset_index(drop = True)

top_users = top_users.sort_values(by='corr', ascending=False)

top_users.rename(columns={"user_id_2": "User-ID"}, inplace=True)
top_users

In [ ]:
top_users_ratings = top_users.merge(ratings[["User-ID","ISBN",
                                            "Book-Rating"]],how ='inner')
top_users_ratings = top_users_ratings[top_users_ratings["User-ID"]
                                      != random_user]

top_users_ratings

In [ ]:
recommendation_df = top_users_ratings.groupby('ISBN').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df

In [ ]:
# We list top 10 with a weighted rating 
books_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 6.5].sort_values("weighted_rating", ascending=False).head(10)

books_to_be_recommend

In [ ]:
# we merge df to see the name of books

books_to_be_recommend.merge(df[["ISBN", "Book-Title"]]).drop_duplicates()